# Model 2 training 



## Before your running

- Open GPU on Colab. Click `Runtime` -> `change runtime type`-> `select GPU`
- Model 2's mask rcnn is runned on detectron2 platform for training. For more details,  please read detectron2 [repo](https://github.com/facebookresearch/detectron2), [tutorial notebook](https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5),[documentation](https://detectron2.readthedocs.io/en/latest/)

## Trained weight of the model
- The trained model weight is saved at `./output/`

# Install detectron2

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
!gcc --version
!pip install gdown
# opencv is pre-installed on colab

# install detectron2: (Colab has CUDA 10.2 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install torch==1.8.0 torchvision==0.9.0 torchaudio==0.8.0
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.8/index.html

1.8.1+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
import torch, torchvision
assert torch.__version__.startswith("1.8")   # need to manually install torch 1.8 if Colab changes its default version

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Train on a building dataset

In this section, we show how to train an existing detectron2 model on a custom dataset in a new format.

We use our building image dataset
which only has 3 class: masonry, m6 and rcw

We'll predict buidling type based on an existing model pre-trained on COCO dataset, available in detectron2's model zoo.

Note that COCO dataset does not have 3 classed. We'll be able to recognize this new class in a few minutes.

## Prepare the dataset

In our mdel, **we use category opening, masonry, m6, rcw**, 

In [ ]:
if not os.path.exists('/content/val_predict'):
  os.makedirs('/content/val_predict')

if not os.path.exists('/content/df'):
  os.makedirs('/content/df')

In [ ]:
# Download buildings imgs data(traning set and val set) from github
! git clone https://github.com/luoyaxiong/final.git  /content/project

fatal: destination path '/content/dataset' already exists and is not an empty directory.


In [ ]:
# address of building dataset
DATASET_ADDRESS = '/content/project/code/data/traing_data/model2'

Register the building dataset to detectron2, following the [detectron2 custom dataset tutorial](https://detectron2.readthedocs.io/tutorials/datasets.html).
Here, the dataset is in its custom format, therefore we write a function to parse it and prepare it into detectron2's standard format. User should write such a function when using a dataset in custom format. See the tutorial for more details.


In [ ]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
# from detectron2.data.datasets import register_coco_instances
# register_coco_instances("my_dataset_train", {}, "json_annotation_train.json", "path/to/image/dir")
# register_coco_instances("my_dataset_val", {}, "json_annotation_val.json", "path/to/image/dir")

from detectron2.structures import BoxMode

def get_building_dicts(img_dir):
    json_file = os.path.join(img_dir, "via_region_data.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)

    dataset_dicts = []
    dict_category_id = {
      'masonry':0,
      'm6':1,
      'rcw':2
    }
    for idx, v in enumerate(imgs_anns.values()):
        record = {}
        
        filename = os.path.join(img_dir, v["filename"])
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
      
        annos = v["regions"]
        objs = []
        # since we use VIA 2.0 tool， annos is a list not a set
        for idx, anno in enumerate(annos):

            # assert not anno["region_attributes"]
            #  record category_id
            category_id = dict_category_id[anno["region_attributes"]['class_name']]
            anno = anno["shape_attributes"]
            px = anno["all_points_x"]
            py = anno["all_points_y"]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": category_id,
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts


In [ ]:
# add the dict to Cataclog
for d in ["train", "val"]:
    DatasetCatalog.register('building_'+d, lambda d=d: get_building_dicts(DATASET_ADDRESS+'/' + d))
    MetadataCatalog.get('building_'+d).set(thing_classes=["masonry","m6","rcw"])
building_metadata = MetadataCatalog.get("building_train")

To verify the data loading is correct, let's visualize the annotations of randomly selected samples in the training set:



## Train

Now, fine-tune a cityscape-pretrained R50-FPN Mask R-CNN model on the building dataset. 


There is no epoch parameter in this frame, but it provides IMS_PER_BATCH and MAX_ITER. We can calculate epoch from equation below:

$$ Epoch = MAX\_ITER * IMS\_PER\_BATCH  / TOTAL\_NUM\_IMAGES 
$$

In [ ]:
EPOCH = 20
TOTAL_NUM_IMAGES = 449

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("building_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = int(EPOCH*TOTAL_NUM_IMAGES/cfg.SOLVER.IMS_PER_BATCH-1)     # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # 3 class ("masonry","m6","rcw"). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

# os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg) 
# trainer.resume_or_load(resume=False)
# trainer.train()

In [ ]:
# Add validation loss 
from detectron2.engine import HookBase
from detectron2.data import build_detection_train_loader
import detectron2.utils.comm as comm

cfg.DATASETS.VAL = ("building_val",)

class ValidationLoss(HookBase):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg.clone()
        self.cfg.DATASETS.TRAIN = cfg.DATASETS.VAL
        self._loader = iter(build_detection_train_loader(self.cfg))
        
    def after_step(self):
        data = next(self._loader)
        with torch.no_grad():
            loss_dict = self.trainer.model(data)
            
            losses = sum(loss_dict.values())
            assert torch.isfinite(losses).all(), loss_dict

            loss_dict_reduced = {"val_" + k: v.item() for k, v in 
                                 comm.reduce_dict(loss_dict).items()}
            losses_reduced = sum(loss for loss in loss_dict_reduced.values())
            if comm.is_main_process():
                self.trainer.storage.put_scalars(total_val_loss=losses_reduced, 
                                                 **loss_dict_reduced)

In [ ]:
# ! rm -r /content/output

In [ ]:
from detectron2.engine import DefaultTrainer
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
val_loss = ValidationLoss(cfg)  
trainer.register_hooks([val_loss])
# swap the order of PeriodicWriter and ValidationLoss
trainer._hooks = trainer._hooks[:-2] + trainer._hooks[-2:][::-1]
trainer.resume_or_load(resume=False)
trainer.train()

[06/14 03:15:14 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (3, 256, 1, 1) 

[06/14 03:15:32 d2.engine.train_loop]: Starting training from iteration 0
[06/14 03:15:38 d2.utils.events]:  eta: 0:12:53  iter: 19  total_loss: 2.187  loss_cls: 1.36  loss_box_reg: 0.01361  loss_mask: 0.6903  loss_rpn_cls: 0.1006  loss_rpn_loc: 0.0112  total_val_loss: 2.192  val_loss_cls: 1.344  val_loss_box_reg: 0.02658  val_loss_mask: 0.6926  val_loss_rpn_cls: 0.08547  val_loss_rpn_loc: 0.01016  time: 0.1739  data_time: 0.0398  lr: 4.9953e-06  max_mem: 2179M
[06/14 03:15:44 d2.utils.events]:  eta: 0:12:15  iter: 39  total_loss: 2.112  loss_cls: 1.223  loss_box_reg: 0.04137  loss_mask: 0.686  loss_rpn_cls: 0.1074  loss_rpn_loc: 0.01084  total_val_loss: 2.061  val_loss_cls: 1.217  val_loss_box_reg: 0.03967  val_loss_mask: 0.6907  val_loss_rpn_cls: 0.09979  val_loss_rpn_loc: 0.01208  time: 0.1714  data_time: 0.0238  lr: 9.9903e-06  max_mem: 2179M
[06/14 03:15:49 d2.utils.events]:  eta: 0:11:48  iter: 59  total_loss: 1.821  loss_cls: 0.984  loss_box_reg: 0.02958  loss_mask: 0.6805  loss

In [ ]:
# Look at training and val curves in tensorboard
%load_ext tensorboard
%tensorboard --logdir output

### Download the weight

In [ ]:
!zip -r /content/weight.zip /content/ouput 

from google.colab import files
files.download("/content/weight.zip") 

## Inference on validation set using the trained model
Now, let's run inference with the trained model on the building validation dataset. First, let's create a predictor using the model we just trained:



In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.80   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

Then, we randomly select several samples to visualize the prediction results.

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_building_dicts(DATASET_ADDRESS+"/val")
# From here to change the numer of imgs to show
# num_to_show = 1
# for d in random.sample(dataset_dicts,num_to_show):  
for d in dataset_dicts[:2]:
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=building_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    img_name = 'predict_'+d["file_name"].split('/')[-1]
    savepath = '/content/val_predict/' + img_name
    cv2.imwrite(savepath, out.get_image()[:, :, ::-1])
    cv2_imshow(out.get_image()[:, :, ::-1])

We can also evaluate its performance using AP metric implemented in COCO API.

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("building_val", ("bbox", "segm"), False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "building_val")
print(inference_on_dataset(trainer.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`

[06/14 03:35:39 d2.evaluation.coco_evaluation]: 'building_val' is not registered by `register_coco_instances`. Therefore trying to convert it to COCO format ...
[06/14 03:35:39 d2.data.datasets.coco]: Converting annotations of dataset 'building_val' to COCO format ...)
[06/14 03:35:41 d2.data.datasets.coco]: Converting dataset dicts into COCO format
[06/14 03:35:41 d2.data.datasets.coco]: Conversion finished, #images: 96, #annotations: 123
[06/14 03:35:42 d2.data.datasets.coco]: Caching COCO format annotations at './output/building_val_coco_format.json' ...
[06/14 03:35:44 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[06/14 03:35:44 d2.data.common]: Serializing 96 elements to byte tensors and concatenating them all ...
[06/14 03:35:44 d2.data.common]: Serialized dataset takes 0.08 MiB
[06/14 03:35:44 d2.evaluation.evaluator]: Start inference on 96 images
[06/14 03:35:47